In [ ]:
import os,sys
sys.path.insert(0, os.path.abspath('..'))
from hublib.ui import FileUpload
from mpl_toolkits.mplot3d import Axes3D

import re

import numpy as np
from scipy import ndimage as ndi


import pandas as pd
from pandas import DataFrame

from scipy.signal import chirp, find_peaks, peak_widths, savgol_filter
from skimage.filters import sobel, gaussian
from skimage import measure

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)
import plotly.io as pio

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

from ipywidgets import FileUpload
import os
import re

import numpy as np
import pandas as pd
from pandas import DataFrame

from scipy import ndimage as ndi
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from scipy.signal import chirp, find_peaks, peak_widths, savgol_filter
from skimage.filters import sobel, gaussian
from skimage import measure

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly import tools
init_notebook_mode(connected=True)


import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
from collections import OrderedDict
from numpy import *
from IPython.display import FileLink, FileLinks, clear_output
from string import Template
from hublib.ui import FileUpload, Download

import os,sys
sys.path.insert(0, os.path.abspath('..'))
from ipywidgets import HBox
from scipy.signal import chirp, find_peaks, peak_widths, savgol_filter
from skimage.filters import sobel, gaussian
from skimage import measure

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)
import plotly.io as pio

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
cmap = plt.get_cmap("tab10")

In [ ]:
def done_cb(w, name):
    # Do something with the files here...
    # We just print their names
    print("%s uploaded" % name)
    
    # reset clears and re-enables the widget for more uploads
    # You may want to do this somewhere else, depending on your GUI
    w.reset()

In [ ]:
fm = FileUpload("TPL files", 
               "This is a description that appears when the mouse hovers over the name.", 
                cb=done_cb, maxnum=100, maxsize='100M')


In [ ]:
text_layout1 = widgets.Layout(width ='145px', position = 'middle')
Designed_Radius= widgets.BoundedFloatText(
    value=2.,
    min=0.1,
    max=3.2,
    step=0.1,
    layout=text_layout1,
    tooltip = 'Designed_Radius'
)

vinputa1 = widgets.VBox([Designed_Radius]) 
#Designed_Radius_label = widgets.Label("Designed Radius (mm):")
dimlabel1 = widgets.HTML('<p style="background:lightgray" align="center"> Radius in Design (&mu;m)</p>')

#vlabelsa1 = widgets.VBox([Designed_Radius_label])
inputs2a1 = widgets.HBox([vinputa1],layout=widgets.Layout(border='none'))


inputs21 = widgets.VBox([inputs2a1])
input2 = widgets.VBox([dimlabel1,inputs21], layout={'width': '150px', 'position': 'middle'})

In [ ]:
#input section
text_layout1 = widgets.Layout(width ='45px',position = 'middle')
column_index= widgets.BoundedFloatText(
    value=1.,
    min=1.,
    max=5.,
    step=1.,
    layout=text_layout1,
    tooltip = 'Column Number'
)

# Laser exposure time
row_index = widgets.BoundedFloatText(
    value = 1.,
    min = 1.,
    max = 5.,
    step = 1.,
    layout=text_layout1,
    tooltip = 'Row number'
)


column_label = widgets.Label("X:")
row_label=widgets.Label(" Y:")
dimlabel = widgets.HTML('<p style="background:lightgray" align="center"> Structure Index</p>')

row_input = widgets.HBox([row_label, row_index]) 
col_input = widgets.HBox([column_label,column_index])
inputs2a = widgets.HBox([col_input, row_input],layout=widgets.Layout(border='none'))


inputs2 = widgets.VBox([inputs2a])
input1 = widgets.VBox([dimlabel,inputs2], layout={'width': '150px'})

In [ ]:
def fitdata_height(fm):
    
    #load data 
    file_list =fm.list()
    file_list = sorted(file_list)

    full_measurement = pd.DataFrame()
    for f in file_list:
        df = pd.read_csv(f)
        full_measurement = full_measurement.append(df)
   

    #Height 
    X = np.array([np.abs(full_measurement.index_x),np.abs(full_measurement.index_y)]).T
    y = full_measurement.height_max_rela.ravel()
    kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
    gp = GaussianProcessRegressor(kernel=kernel, alpha=0.01,
                                  n_restarts_optimizer=10)
    gp.fit(X, y)
    desired_height =Designed_Radius.value
    height_factor = gp.predict(X[:25,:])/desired_height
    
    
   
    return height_factor

In [ ]:
def fitdata_radius(fm):
       #load data 
    file_list =fm.list()
    file_list = sorted(file_list)

    full_measurement = pd.DataFrame()
    for f in file_list:
        df = pd.read_csv(f)
        full_measurement = full_measurement.append(df)
     #Radius 
    X = np.array([np.abs(full_measurement.index_x),np.abs(full_measurement.index_y)]).T
    y = full_measurement.equivalent_diameter.ravel()/2
    kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
    gp = GaussianProcessRegressor(kernel=kernel, alpha=0.01,
                                  n_restarts_optimizer=10)
    gp.fit(X, y)
    desired_radius = Designed_Radius.value
    radius_factor = gp.predict(X[:25,:])/desired_radius
    
    return radius_factor

In [ ]:
def plot_figure(fm,height_factor,radius_factor,out):
    #plot two figures
    feature_df = pd.read_csv(fm.list()[0]) 
   
    trace1 = go.Scatter(
        x = feature_df['index_x'],
        y = feature_df['index_y'],
        mode='markers',
        marker=dict(
            size = MinMaxScaler((15, 30)).fit_transform(
                Designed_Radius.value/height_factor.reshape(-1,1)
            ).reshape(-1),
            color = 2/height_factor,
            colorscale='Viridis',
            colorbar=dict(
                x = 0.4,
                y = 0.5,
                lenmode='fraction', 
                len=0.83,
            ),
            showscale=True
        )
    )
    
    
    trace2 = go.Scatter(
        x = feature_df['index_x'],
        y = feature_df['index_y'],
        mode='markers',
        marker=dict(
            size = MinMaxScaler((15, 30)).fit_transform(
                 Designed_Radius.value/radius_factor.reshape(-1,1)
            ).reshape(-1),
            color = 2/radius_factor, 
            colorscale='Viridis',
            colorbar=dict(
                x = 1,
                y = 0.5,
                lenmode='fraction', 
                len=0.83,
            ),
            showscale=True
        ),
    )
    
    
    fig = make_subplots(rows=1, cols=2, subplot_titles = [
        'Height Compensation', 'Radius Compensation',
  
    ])
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=1, col=2)

    
    fig.update_xaxes(title_text="X axis", domain=[0, 0.4], tick0 = 1, dtick = 1, row=1, col=1)
    fig.update_xaxes(title_text="X axis", domain=[0.6, 1], tick0 = 1, dtick = 1, row=1, col=2)


    fig.update_yaxes(title_text="Y axis", tick0 = 1, dtick = 1, row=1, col=1)
    fig.update_yaxes(title_text="Y axis", tick0 = 1, dtick = 1, row=1, col=2)

    
    fig.update_layout(height=450, width = 900, showlegend=False,)

    iplot(fig)

In [ ]:
def save_file(fm,height_factor,radius_factor,out):
    filename=fm.list()[0]
    label=np.arange(1,26)
    
    y = (label - 1) // 5+1
    x=(label-(y-1)*5)
    compensation_height =Designed_Radius.value/height_factor.reshape(-1,1)
    compensation_radius =Designed_Radius.value/radius_factor.reshape(-1,1)
    data = {
            'index': label,
            'index_x': np.reshape(x,-1),
            'index_y': np.reshape(y,-1),
            'Compensated Height':np.reshape(compensation_height,-1),
            'Compensated Radius':np.reshape(compensation_radius,-1),
            'Height Factor':np.reshape(height_factor,-1),
            'Radius Factor':np.reshape(radius_factor,-1)
    }
    df = DataFrame(data)
    
    feature_file_path = 'download/' + 'Compensation.csv'
    df.to_csv(feature_file_path, index = False)

    return feature_file_path

In [ ]:
def measurement_analystic(fm,out):
    height_factor=fitdata_height(fm)
    radius_factor=fitdata_radius(fm)
    plot_figure(fm,height_factor,radius_factor,out)
    new_file=save_file(fm,height_factor,radius_factor,out)
    
    

    
    filename ='download/'+'Compensation.csv'
    Download(filename, style='info')

In [ ]:
def Visualization(fm,out):
     
    #load data 
    file_list =fm.list()
    file_list = sorted(file_list)

    full_measurement = pd.DataFrame()
    for f in file_list:
        df = pd.read_csv(f)
        full_measurement = full_measurement.append(df)
   

    #Height 
    X = np.array([np.abs(full_measurement.index_x),np.abs(full_measurement.index_y)]).T
    y = full_measurement.height_max_rela.ravel()
    kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
    gp = GaussianProcessRegressor(kernel=kernel, alpha=0.01,
                                  n_restarts_optimizer=10)
    gp.fit(X, y)
    desired_height =Designed_Radius.value
    height_factor = gp.predict(X[:25,:])/desired_height
    
    
    #Radius 
    X = np.array([np.abs(full_measurement.index_x),np.abs(full_measurement.index_y)]).T
    y = full_measurement.equivalent_diameter.ravel()/2
    kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
    gp = GaussianProcessRegressor(kernel=kernel, alpha=0.01,
                                  n_restarts_optimizer=10)
    gp.fit(X, y)
    desired_radius = Designed_Radius.value
    radius_factor = gp.predict(X[:25,:])/desired_radius
    
    index=(row_index.value-1)*5 +column_index.value-1
    
    u, v = np.mgrid[0:2*np.pi:50j, 0:0.5*np.pi:50j]
    x = desired_radius/radius_factor[int(index)]*np.cos(u)*np.sin(v)
    y = desired_radius/radius_factor[int(index)]*np.sin(u)*np.sin(v)
    z = desired_height/height_factor[int(index)]*np.cos(v)

    XX=desired_radius*np.cos(u)*np.sin(v)
    YY=desired_radius*np.sin(u)*np.sin(v)
    ZZ=desired_height*np.cos(v)
    

    
    trace1 = go.Surface(x = x, y = y, z=z,colorbar=None, showscale=False, opacity=0.2)
    trace2 = go.Surface(x = XX, y = YY, z=ZZ,colorscale='Blues', showscale=False, opacity=1)
    
    data = [trace1, trace2]

    layout = go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        scene = dict(
            xaxis=dict(
                title='X (' + u"\u03BC" + 'm)'
            ),
            yaxis=dict(
                title='Y (' + u"\u03BC" + 'm)'),
            zaxis = dict(
                title='Z (' + u"\u03BC" + 'm)'),
            aspectmode='data',
            camera = dict(
                eye=dict(x=2, y=-2, z=1)),
        ),

    )

    fig = dict(data = data, layout = layout)
    iplot(fig)

    plt.show()


In [ ]:
button1 = widgets.Button(description="Analyze", layout={'width': '145px'})
output1 = widgets.Output(layout={'border': '1px solid green', 
                                 'min_width': '920px', 'min_height': '500px',})


def on_button_clicked_1(a):
     with output1:
        clear_output()
        measurement_analystic(fm,output1)
        filename ='download/'+'Compensation.csv'
        save=Download(filename, style='info')
        display(save)

    
button2 = widgets.Button(description="Visualize", layout={'width': '145px'})

def on_button_clicked_2(a):
     with output1:
        clear_output()
        Visualization(fm,output1)
        
#display part:
display(fm)


#display(button1, output1)
button1.on_click(on_button_clicked_1)
button2.on_click(on_button_clicked_2)
left = widgets.VBox([input2,button1], layout={'min_width': '140px', 'position':'middle'})
middle = widgets.VBox([input1,button2], layout={'min_width': '110px', 'position':'middle'})
upper_box = widgets.HBox([left, middle], layout= {'justify_content': 'flex-start'})
widgets.VBox([upper_box,output1], layout= {'justify_content': 'flex-start'})